In [28]:
# Model name

# model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
#  1.5 sek
#  Engelsk
#  1/ 2 poeng
#  Norsk
#  2/ 2 poeng


# model_name = 'ltg/norbert3-large'
#  2.3 sek
#  Engelsk
#  0?/ 2 poeng
#  Norsk
#  0?/ 2 poeng

model_name = 'microsoft/DialoGPT-small'
#  0 sek
#  Engelsk
#  0/ 2 poeng
#  Norsk
#  0/ 2 poeng




# model_name = 'lmsys/vicuna-13b-v1.5-16k'
#   sek
#  Engelsk
#  0?/ 2 poeng
#  Norsk
#  0?/ 2 poeng

# model_name = 'pankajmathur/orca_mini_v3_7b'
#   sek
#  Engelsk
#  0?/ 2 poeng
#  Norsk
#  0?/ 2 poeng



In [32]:
from transformers import AlbertTokenizer, AlbertForQuestionAnswering, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM
import torch
# model = AutoModelForQuestionAnswering.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,  trust_remote_code=True)


generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 640kB/s]


In [33]:
def get_answer(text, question):
    # Tokenize the input text and question
    inputs = tokenizer(question, text, return_tensors='pt', max_length=512, truncation=True)
    outputs = model(**inputs)
    # Get the start and end scores for the answer
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
    return answer





In [66]:
# Engelsk
context = """
Stig is the boss at Aboveit. He is a very capable boss, and sometimes he laughs at jokes.
"""
question = "Hello, who is the boss at Aboveit?"
answer = get_answer(context, question)
print(f"SHOULD BE 'Stig':        {answer}")

context = """
Stig is not the boss at Aboveit. He is a very capable boss, and sometimes he laughs at jokes.
"""
question = "Hello, who is the boss at Aboveit?"
answer = get_answer(context, question)
print(f"SHOULD NOT BE 'Stig':    {answer}")


context = """
Stig er sjef i Aboveit. Han er en veldig dyktig sjef, og av og til ler han av vitser.
"""
question = "Hei, hvem er sjefen i Aboveit?"
answer = get_answer(context, question)
print(f"Bør være 'Stig':         {answer}")

context = """
Stig er ikke sjef i Aboveit. Han er en veldig dyktig sjef, og av og til ler han av vitser.
"""
question = "Hei, hvem er sjefen i Aboveit?"
answer = get_answer(context, question)
print(f"Bør ikke være 'Stig':    {answer}")





SHOULD BE 'Stig':        sometimes he
SHOULD NOT BE 'Stig':    sometimes he
Bør være 'Stig':         
Bør ikke være 'Stig':    
